In [1]:
import pandas as pd
from utils.connect import GetQuery

Currently we have 29432 securities on the platform and calculate riskscore for 15682 fo them


In [2]:
all_tickers = GetQuery(f"""
SELECT DISTINCT(symbol) from base_tickers
""").symbol

tickers_risk = GetQuery(f"""
SELECT DISTINCT(symbol) from ticker_risk_scores
""").symbol

In [3]:
len(all_tickers)

29517

In [4]:
len(tickers_risk)

15716

## Proposed change

Have all tickers for information (search only). Calculate risk/matchscore only for tickers with the following characteristics

**For stocks**
- Market Cap > $100mln

- Avg 90-day Daily volume > $500K (dollars not # of shares)


**For ETFs**
- NAV over 100 mln (check #)

- Avg 90-day Daily volume > $500K (dollars not # of shares)


**For Crypto**

- Top 100 by market cap rank

- Avg 90-day Daily volume > $500K (dollars not # of shares)

In [5]:
def FilteredStocks():
    # Get Stocks only
    tickers = GetQuery("""
    SELECT bt.symbol, bt.type, tm.market_capitalization, tm.avg_volume_90d_money 
    FROM base_tickers bt
    LEFT JOIN ticker_metrics tm
    ON bt.symbol=tm.symbol
    WHERE bt.type IN ('common stock', 'preferred stock') AND
    tm.market_capitalization>=100000000 AND tm.avg_volume_90d_money > 500000 
    
    """)
    
    return tickers

In [6]:
stocks = FilteredStocks()
stocks

,symbol,type,market_capitalization,avg_volume_90d_money
0,QSR,common stock,29205020672,1.023580e+08
1,GHC,common stock,2888666624,1.173312e+07
2,MARA,common stock,453346976,1.663966e+08
3,GH,common stock,3079226112,5.885755e+07
4,KW,common stock,2170208256,1.032229e+07
...,...,...,...,...
4265,PETQ,common stock,299328128,4.922947e+06
4266,TMUS,common stock,177117200384,8.045765e+08
4267,NEU,common stock,2995883264,1.401760e+07
4268,PAXS,common stock,675578880,2.983109e+06


In [7]:
# Filter ETFs

def FilterETF():
    
    # Get ETFs and Funds only
    tickers = GetQuery("""
    SELECT bt.symbol, bt.type, tm.avg_volume_90d_money 
    FROM base_tickers bt
    LEFT JOIN ticker_metrics tm
    ON bt.symbol=tm.symbol
    WHERE bt.type IN ('etf', 'fund', 'mutual fund') 
    AND tm.avg_volume_90d_money > 500000
    """)
    
    return tickers

In [8]:
etfs = FilterETF()
etfs

,symbol,type,avg_volume_90d_money
0,FDRR,etf,1.886861e+06
1,TSLQ,etf,6.736017e+07
2,FEM,etf,3.105153e+06
3,FLCB,etf,2.406972e+06
4,FLN,etf,6.108756e+05
...,...,...,...
1652,DCPE,etf,2.917529e+06
1653,DFSE,etf,2.190555e+06
1654,DBP,etf,1.305319e+06
1655,DFCF,etf,2.894085e+07


In [9]:
def FilterCoins():
    # Get ETFs and Funds only
    tickers = GetQuery("""
    SELECT m.symbol, m.type, m.avg_volume_90d_money, c.market_cap_rank
    FROM (SELECT bt.symbol, bt.type, tm.avg_volume_90d_money 
        FROM base_tickers bt
        LEFT JOIN ticker_metrics tm
        ON bt.symbol=tm.symbol
        WHERE bt.type = 'crypto' 
        AND tm.avg_volume_90d_money > 500000) m
    LEFT JOIN coingecko_coin c 
    ON REPLACE(m.symbol,'.CC','')=UPPER(c.symbol)
    WHERE c.market_cap_rank <=100 
    AND m.avg_volume_90d_money >= 500000
    

    """)
    
    return tickers

In [10]:
coins = FilterCoins()
coins

,symbol,type,avg_volume_90d_money,market_cap_rank
0,HBAR.CC,crypto,2.013585e+06,36.0
1,ICP.CC,crypto,1.895453e+08,39.0
2,HT.CC,crypto,2.036143e+08,43.0
3,XMR.CC,crypto,1.191383e+10,25.0
4,KCS.CC,crypto,1.769104e+07,56.0
...,...,...,...,...
75,CRO.CC,crypto,3.584815e+06,31.0
76,CRV.CC,crypto,2.530076e+06,93.0
77,EGLD.CC,crypto,2.257713e+09,44.0
78,EOS.CC,crypto,1.849644e+08,40.0


In [11]:
GetQuery(f"""
SELECT symbol, risk_score FROM ticker_risk_scores WHERE
symbol IN {tuple(coins.symbol)} AND risk_score<=0.8
""").sort_values('risk_score')

,symbol,risk_score
19,DAI.CC,0.226703
5,USDT.CC,0.226703
22,TUSD.CC,0.226703
21,USDC.CC,0.226703
20,BUSD.CC,0.226703
27,GUSD.CC,0.227974
29,USDP.CC,0.228228
28,FRAX.CC,0.229244
24,XAUT.CC,0.230007
0,PAXG.CC,0.230007


## Conclusion

We will have 6011 securities with match scores on the platform

- Stocks: 4284

- ETFs and funds: 1547

- Crypto: 80

In [12]:
all_risk_ranked = pd.concat([
    stocks[['symbol','type']],
    etfs[['symbol', 'type']],
    coins[['symbol', 'type']]
], ignore_index=True)

In [13]:
all_risk_ranked.shape

(6007, 2)

In [14]:
all_risk_ranked.groupby('type').symbol.count()

type
common stock       4195
crypto               80
etf                1442
fund                215
preferred stock      75
Name: symbol, dtype: int64

In [15]:
all_risk_ranked.to_csv('./list_ranked.csv')

## Check if tradable

The ones that we can trade will also have 'Invest' button

In [16]:
trade = pd.read_csv("./Instruments List_PROD_NOV2022.csv")

In [17]:
tradable_not_ranked = list(set(trade.Symbol) - set(all_risk_ranked.symbol))

In [18]:
len(tradable_not_ranked)

2116

In [19]:
tradable_not_ranked[:30]

['FBRX',
 'OWLT',
 'KLTR',
 'GXG',
 'FMITX',
 'EZU',
 'BGLD',
 'CIBFX',
 'HELX',
 'KBND',
 'BALT',
 'PTTAX',
 'CLIX',
 'VWNFX',
 'IQSU',
 'CMMB',
 'BJBHX',
 'ELVT',
 'EAOK',
 'VHAQ',
 'EVIBX',
 'VMLUX',
 'GSID',
 'HYUP',
 'FDEC',
 'FAUG',
 'SGENX',
 'LZAGY',
 'AGLE',
 'FKGRX']

In [20]:
ranked_not_tradable = list(set(all_risk_ranked.symbol.str.replace("-",".")) - set(trade.Symbol))
len(ranked_not_tradable)

707

In [21]:
ranked_not_tradable[:30]

['JKJ',
 'FLYA',
 'XL',
 'EOCT',
 'RF.PC',
 'FVAM',
 'USCT',
 'RFDA',
 'YALL',
 'ETHW.CC',
 'BTSE.CC',
 'JILL',
 'JPSE',
 'BKIE',
 'ATOM.CC',
 'SLCR',
 'CVET',
 'NVBW',
 'NBCM',
 'JKK',
 'THCP',
 'IACC',
 'ROLL',
 'AMPY',
 'NBH',
 'WFC.PY',
 'DFSE',
 'QRHC',
 'CGABL',
 'CMLS']

In [22]:
ranked_and_tradable = [v for v in all_risk_ranked.symbol.str.replace("-",".") if v in trade.Symbol.values]

In [23]:
len(ranked_and_tradable)

5300

In [24]:
# Check that all TTF stocks have a rank

In [25]:
ttfs = GetQuery("""
SELECT distinct(symbol) FROM collection_ticker_weights
WHERE date::date>='2022-12-01' AND weight IS NOT NULL
""")

In [26]:
set(ttfs.symbol.str.replace("-",".")) - set(ranked_and_tradable)

{'AGFY',
 'ARBK',
 'FTEK',
 'HNRG',
 'IREN',
 'LTCH',
 'SDIG',
 'SRS',
 'USX',
 'VLTA',
 'VORB',
 'WRE',
 'WULF'}

In [27]:
set(ttfs.symbol.str.replace("-",".")) - set(trade.Symbol)

{'ARBK', 'FTEK', 'HNRG', 'SRS', 'WRE', 'WULF'}